In [1]:
# %%javascript
# IPython.OutputArea.auto_scroll_threshold = 9999;

In [2]:
import datetime
import os
import matplotlib
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import os
import blosc

from notebook_shared import decomposeutils as dcu
from notebook_shared import utils

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

from pandas import Series

import seaborn as sns
sns.set()
matplotlib.style.use('seaborn-colorblind')

INPUT_FILE = "dataset"
FSIZE = "tiny" #"tiny", "small" or "full"

df = pd.read_parquet(utils.get_dataset_path(INPUT_FILE, FSIZE))

In [3]:
from IPython.display import display

def output_decompose(provider, region, dt_rounding, decompose_result):
    display(decompose_result['decomposition']['fig'])
    display(decompose_result['seasonal_analysis']['fig'])
    print(decompose_result['stats_test']['case_desc'])
    print(decompose_result['stats_test']['adfuller']['result']['text'])
    print(decompose_result['stats_test']['kpss']['result']['text'])
    display(decompose_result['stats_test']['fig_corr'])
    display(decompose_result['stats_test']['fig_dist'])

In [4]:
def save_decompose(label, decompose_result):
    
    provider = decompose_result['parameters']['provider']
    region = decompose_result['parameters']['region']
    decompose_col = decompose_result['parameters']['decompose_col']
    
    if decompose_result['stats_test'] is None:
        utils.savefig(
            decompose_result['decomposition']['fig'],
            ['temporal_analysis', 'decomposition', decompose_col, provider, f'{region}_{label}']
        )
    else:            
        utils.savefig(
            decompose_result['decomposition']['fig'],
            ['temporal_analysis', 'decomposition', decompose_col, provider, f'{region}_{label}'],
            case=decompose_result['stats_test']['case_desc'],
            adfuller=decompose_result['stats_test']['adfuller']['result']['text'],
            kpss=decompose_result['stats_test']['kpss']['result']['text'])

        utils.savefig(
            decompose_result['stats_test']['fig_dist'],
            ['temporal_analysis', 'decomposition', decompose_col, provider, f'{region}_{label}_dist']
        )

        utils.savefig(
            decompose_result['stats_test']['fig_dist'],
            ['temporal_analysis', 'decomposition', decompose_col, provider, f'{region}_{label}_corr']
        )

    utils.savefig(
            decompose_result['seasonal_analysis']['fig'],
            ['temporal_analysis', 'decomposition', decompose_col, provider, f'{region}_{label}_seasonal']
        )

In [5]:
def strip_for_pickle(decompose_result):

    for result in decompose_result:
        result['decomposition']['fig'] = None
    
    if result['stats_test'] is not None:
        result['stats_test']['fig_dist'] = None
        result['seasonal_analysis']['fig'] = None
    return decompose_result

# Decomposition Runtime Half-Hourly, Hourly, Daily

In [6]:
decompositions_params = [
    {"label" : 'half-hourly', "dt_rounding" : '30min', "adflag" : 48, 'dtfmt':'%d/%m (%a)',  "providers" : ['AWS','AZURE', 'GCP'] },
    {"label" : 'hourly',      "dt_rounding" : 'H',     "adflag" : 24, 'dtfmt':'%d/%m (%a)',  "providers" : ['AWS','AZURE', 'GCP'] },
    {"label" : 'daily',       "dt_rounding" : 'D',     "adflag" : 7,  'dtfmt':'%d/%m (%a)',  "providers" : ['AWS','AZURE', 'GCP'] }
]

In [7]:
# %matplotlib agg
from concurrent.futures import ThreadPoolExecutor 
import warnings
from statsmodels.tools.sm_exceptions import InterpolationWarning
warnings.simplefilter('ignore', InterpolationWarning)
import multiprocessing

decompose_runtime_results = {}
cpus = 4 # otherwise memory dies

sns.set(font_scale=1.5)
decomp_dataset = df
for decomp_param in decompositions_params:
    label, dt_rounding, adflag, dtfmt, providers = decomp_param.values()
    decompositions = []
    for provider in providers:
        regions = list(decomp_dataset[decomp_dataset['provider'] == provider].groupby(['provider'], observed=True)['region'].unique().values[0])
        
        def process(region):
            decomp = dcu.decompose(decomp_dataset, provider, region, 'runtime', dt_rounding, adflag=adflag, test_regression='ct', dtfmt=dtfmt)
            decompositions.append(decomp)
            return region
        
        with ThreadPoolExecutor(max_workers=cpus) as executor:
            results = list(executor.map(process, regions))
            
        executor.shutdown(wait=True)
        print('processed:', provider, results)
    decompose_runtime_results[label] = decompositions
# %matplotlib inline

processed: AWS ['ap-northeast-1', 'ap-south-1', 'ap-southeast-2', 'ca-central-1', 'eu-central-1', 'eu-west-2', 'sa-east-1', 'us-east-1', 'us-west-1', 'us-west-2']
processed: AZURE ['australiaeast', 'brazilsouth', 'centralindia', 'eastus', 'germanywestcentral', 'japaneast', 'uksouth', 'westus', 'westus2']
processed: GCP ['asia-northeast1', 'asia-south1', 'australia-southeast1', 'europe-west2', 'europe-west3', 'northamerica-northeast1', 'southamerica-east1', 'us-east4', 'us-west2', 'us-west4']
processed: AWS ['ap-northeast-1', 'ap-south-1', 'ap-southeast-2', 'ca-central-1', 'eu-central-1', 'eu-west-2', 'sa-east-1', 'us-east-1', 'us-west-1', 'us-west-2']
processed: AZURE ['australiaeast', 'brazilsouth', 'centralindia', 'eastus', 'germanywestcentral', 'japaneast', 'uksouth', 'westus', 'westus2']
processed: GCP ['asia-northeast1', 'asia-south1', 'australia-southeast1', 'europe-west2', 'europe-west3', 'northamerica-northeast1', 'southamerica-east1', 'us-east4', 'us-west2', 'us-west4']
proces

In [8]:
fdir = os.path.join('pickles', 'temporal_analysis')
os.makedirs(fdir, exist_ok=True)

labels = ['half-hourly', 'hourly', 'daily']
for label in labels:
    decompose_results = decompose_runtime_results[label]
    decompose_col = decompose_results[0]['parameters']['decompose_col'] # not ideal, but ok
    for results in decompose_results:
        save_decompose(label, results)
    decompose_results = strip_for_pickle(decompose_results)
    pickled_data = pickle.dumps(decompose_results, pickle.HIGHEST_PROTOCOL)
    compressed_pickle = blosc.compress(pickled_data)
    with open(os.path.join(fdir, f'decomposition-{decompose_col}-{FSIZE}-{label}.pc'), 'wb') as f:
        f.write(compressed_pickle)

# Decomposition No CPU per Invocation Half-Hourly, Hourly, Daily

In [9]:
# Create CPU Counts
df_cpus = df[['provider', 'region', 'timezone', 'cpuType', 'workload_invocation', 'driver_invocation']]
df_cpu_counts = df_cpus.groupby(['driver_invocation','provider', 'region', 'timezone', 'workload_invocation'], observed=True)['cpuType'].nunique().rename('NoCpuTypeInvocation')
df_cpu_counts = pd.DataFrame(df_cpu_counts).reset_index()
df_cpu_counts.groupby(['provider', 'region'], observed=True).describe()

NoCpuTypeInvocation                           \
                                               count      mean       std  min   
provider region                                                                 
AWS      ap-northeast-1                       2895.0  1.013126  0.113834  1.0   
         ap-south-1                           2894.0  1.038701  0.192914  1.0   
         ap-southeast-2                       2896.0  1.027970  0.164914  1.0   
         ca-central-1                         2897.0  1.066275  0.248806  1.0   
         eu-central-1                         2895.0  1.008636  0.092542  1.0   
         eu-west-2                            2893.0  1.001728  0.041544  1.0   
         sa-east-1                            2894.0  1.018659  0.135342  1.0   
         us-east-1                            2885.0  1.134835  0.341607  1.0   
         us-west-1                            2893.0  1.081922  0.274293  1.0   
         us-west-2                            2886.0  1.001386  0.037210  1.0   
AZURE    australiaeast                        2875.0  1.661565  0.608940  1.0   
         brazilsouth                          2878.0  1.773454  0.614474  1.0   
         centralindia                         2895.0  1.338515  0.476923  1.0   
         eastus                               2878.0  1.934677  0.588890  1.0   
         germanywestcentral                   2898.0  1.489303  0.516276  1.0   
         japaneast                            2896.0  1.747238  0.595022  1.0   
         uksouth                              2897.0  1.751467  0.609279  1.0   
         westus                               2836.0  1.820522  0.594402  1.0   
         westus2                              2896.0  1.194061  0.424198  1.0   
GCP      asia-northeast1                      2900.0  1.953793  0.625934  1.0   
         asia-south1                          2901.0  1.000000  0.000000  1.0   
         australia-southeast1                 2901.0  1.802827  0.397933  1.0   
         europe-west2                         2901.0  1.084454  0.278115  1.0   
         europe-west3                         2901.0  1.685281  0.464484  1.0   
         northamerica-northeast1              2901.0  1.000000  0.000000  1.0   
         southamerica-east1                   2900.0  1.860000  0.347047  1.0   
         us-east4                             2901.0  1.635643  0.481332  1.0   
         us-west2                             2900.0  1.000000  0.000000  1.0   
         us-west4                             2900.0  1.000000  0.000000  1.0   

                                                      
                                  25%  50%  75%  max  
provider region                                       
AWS      ap-northeast-1           1.0  1.0  1.0  2.0  
         ap-south-1               1.0  1.0  1.0  2.0  
         ap-southeast-2           1.0  1.0  1.0  2.0  
         ca-central-1             1.0  1.0  1.0  2.0  
         eu-central-1             1.0  1.0  1.0  2.0  
         eu-west-2                1.0  1.0  1.0  2.0  
         sa-east-1                1.0  1.0  1.0  2.0  
         us-east-1                1.0  1.0  1.0  2.0  
         us-west-1                1.0  1.0  1.0  2.0  
         us-west-2                1.0  1.0  1.0  2.0  
AZURE    australiaeast            1.0  2.0  2.0  3.0  
         brazilsouth              1.0  2.0  2.0  4.0  
         centralindia             1.0  1.0  2.0  3.0  
         eastus                   2.0  2.0  2.0  3.0  
         germanywestcentral       1.0  1.0  2.0  3.0  
         japaneast                1.0  2.0  2.0  3.0  
         uksouth                  1.0  2.0  2.0  4.0  
         westus                   1.0  2.0  2.0  3.0  
         westus2                  1.0  1.0  1.0  3.0  
GCP      asia-northeast1          2.0  2.0  2.0  3.0  
         asia-south1              1.0  1.0  1.0  1.0  
         australia-southeast1     2.0  2.0  2.0  2.0  
         europe-west2             1.0  1.0  1.0  2.0  
         e

In [10]:
%matplotlib agg
from concurrent.futures import ThreadPoolExecutor 
import warnings
from statsmodels.tools.sm_exceptions import InterpolationWarning
warnings.simplefilter('ignore', InterpolationWarning)
import multiprocessing

decompose_cpuinvo_results = {}
cpus = 4 # otherwise memory dies
sns.set(font_scale=1.5)

decomp_dataset = df_cpu_counts
for decomp_param in decompositions_params:
    label, dt_rounding, adflag, dtfmt, providers = decomp_param.values()
    decompositions = []
    for provider in providers:
        regions = list(decomp_dataset[decomp_dataset['provider'] == provider].groupby(['provider'], observed=True)['region'].unique().values[0])
        
        def process(region):
            decomp = dcu.decompose(decomp_dataset, provider, region, 'NoCpuTypeInvocation', dt_rounding, 
                                   adflag=adflag, test_regression='ct', dtfmt=dtfmt,
                                   statsTest = False, agg='mean')
            decompositions.append(decomp)
            return region
        
        with ThreadPoolExecutor(max_workers=cpus) as executor:
            results = list(executor.map(process, regions))
            
        executor.shutdown(wait=True)
        print('processed:', provider, results)
    decompose_cpuinvo_results[label] = decompositions
%matplotlib inline

processed: AWS ['ap-northeast-1', 'ap-south-1', 'ap-southeast-2', 'ca-central-1', 'eu-central-1', 'eu-west-2', 'sa-east-1', 'us-east-1', 'us-west-1', 'us-west-2']
processed: AZURE ['australiaeast', 'brazilsouth', 'centralindia', 'eastus', 'germanywestcentral', 'japaneast', 'uksouth', 'westus', 'westus2']
processed: GCP ['asia-northeast1', 'asia-south1', 'australia-southeast1', 'europe-west2', 'europe-west3', 'northamerica-northeast1', 'southamerica-east1', 'us-east4', 'us-west2', 'us-west4']
processed: AWS ['ap-northeast-1', 'ap-south-1', 'ap-southeast-2', 'ca-central-1', 'eu-central-1', 'eu-west-2', 'sa-east-1', 'us-east-1', 'us-west-1', 'us-west-2']
processed: AZURE ['australiaeast', 'brazilsouth', 'centralindia', 'eastus', 'germanywestcentral', 'japaneast', 'uksouth', 'westus', 'westus2']
processed: GCP ['asia-northeast1', 'asia-south1', 'australia-southeast1', 'europe-west2', 'europe-west3', 'northamerica-northeast1', 'southamerica-east1', 'us-east4', 'us-west2', 'us-west4']
proces

In [11]:
fdir = os.path.join('pickles', 'temporal_analysis')
os.makedirs(fdir, exist_ok=True)

labels = ['half-hourly', 'hourly', 'daily']
for label in labels:
    decompose_results = decompose_cpuinvo_results[label]
    decompose_col = decompose_results[0]['parameters']['decompose_col'] # not ideal, but ok
    for results in decompose_results:
        save_decompose(label, results)
    decompose_results = strip_for_pickle(decompose_results)
    pickled_data = pickle.dumps(decompose_results, pickle.HIGHEST_PROTOCOL)
    compressed_pickle = blosc.compress(pickled_data)
    with open(os.path.join(fdir, f'decomposition-{decompose_col}-{FSIZE}-{label}.pc'), 'wb') as f:
        f.write(compressed_pickle)